### Почта для сдачи решений:  
    bdt-mf-ml-nlp-2020-q4@bigdatateam.org

**В теме письма укажите:**
    
    "HW2:CompLing. ФИО"

### Пожалуйста, оцените качество заданий второго модуля [здесь](http://rebrand.ly/mfnlp2020q4_feedback_hw02)
-----------------------------------------------------------------------------------------------------------------

# Dependency Parsing

Мы будем использовать **POS tagging** и **dependency parsing** из популярного пакета [SpaCy](https://spacy.io)

В SpaCy используется подход Universal Dependencies

## Установим модель SpaCy

В SpaCy есть возможность не только размечать части речи и синтаксическиезависимости, но и красиво визуализировать, используя ``displacy``

In [1]:
import spacy
from spacy import displacy

In [2]:
nlp = spacy.load('en')
test_doc = nlp('This is a test.')
displacy.render(test_doc, jupyter = True)

### Spacy сама токенизирует текст, все токены становятся специальными элементами, в которых хранятся сам токен,а также его таги

In [3]:
print([(token, token.pos_) for token in test_doc])

[(This, 'DET'), (is, 'AUX'), (a, 'DET'), (test, 'NOUN'), (., 'PUNCT')]


In [4]:
print(test_doc)

This is a test.


### Для задания мы используем известный корпус литературных произведений - Project Gutenberg. 
Из корпуса мы возьмем довольно длинный роман Джейн Остин "Чувство и чувствительность" (Sense and Sensibility)

In [5]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
sentences = gutenberg.sents('austen-sense.txt')

[nltk_data] Downloading package gutenberg to /home/variya/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [7]:
# import nltk
# nltk.download('punkt')

### Вопросы

**1. Сколько предложений в этом романе? Сколько уникальных токенов?**

In [8]:
type(sentences)

nltk.corpus.reader.util.StreamBackedCorpusView

In [9]:
# Сколько предложений в этом романе?
len(sentences)

4999

In [10]:
vocab = set([token.lower() for sentence in sentences for token in sentence])

In [11]:
#Сколько уникальных токенов?
len(vocab)

6398

**2. Найдите 5 наиболее частотных глагольных словоформ? Найдите 5 наиболее частотных лемм (начальных форм) глаголов? (глагол-verb)**

In [12]:
from collections import Counter

In [13]:
tokens = nlp(' '.join([token.lower() for sentence in sentences for token in sentence]))

In [14]:
word_verb = [token for token in tokens if token.pos_ == 'VERB']

In [15]:
Counter(word_verb).most_common(5)

[(settled, 1), (lived, 1), (engage, 1), (surrounding, 1), (lived, 1)]

In [16]:
lemma_verb = [token.lemma_ for token in tokens if token.pos_ == 'VERB']

In [17]:
Counter(lemma_verb).most_common(5)

[('say', 609), ('could', 578), ('would', 515), ('may', 390), ('see', 384)]

## Грамматика зависимостей (Dependency parsing) and PP attachment

Теперь посмотрим как глаголы и предложные фразы (prepositional phrase или PP) могут быть связаны в предложении.
- Вершина *предложной фразы* - предлог. Например, "**in** the house", "**on** the table", "**with** my friend"


### Вопросы

  3. В чем отличие в зависимости предложных фраз между предложениями группы (A) и предложениями группы (B)? 
  Посмотрите на схему зависимостей, которую можно сделать с помощью SpaCy.
  - Подсказка: какие "родительские" вершины у предложных групп?
  
(A)
  * I eat an apple in my room.
  * We listen to music at the theater.
  * John visited Brazil with his friend.
  
(B)
  * I see a fly in my soup.
  * She knows the man at the store.
  * I photographed a man with a bowtie.
  

In [18]:
sentance_groups = {'A': ['I eat an apple in my room.', 
                         'We listen to music at the theater.',
                         'John visited Brazil with his friend.'],
                   'B': ['I see a fly in my soup.', 
                         'She knows the man at the store.',
                         'I photographed a man with a bowtie.']}

In [19]:
def show_linkage(text):
    doc = nlp(text)
    displacy.render(doc, jupyter=True)

In [20]:
# A
for sent in sentance_groups['A']:
    show_linkage(sent)

In [21]:
# B
for sent in sentance_groups['B']:
    show_linkage(sent)

**Вывод**: в группе А предложные фразы зависят от глагола, который выполняет роль сказуемого, а в группе В предложные фразы зависят от дополнения, кроме последнего предложения

### Бывают случаи, когда не очевидно, к какому элементу предложения нужно присоединить предложную фразу
- Вспомните пример из лекции "I shot an elephant in my pyjamas"

На примере нашего корпуса попробуем понять, какую "родительскую" вершину для предложных фраз предпочитает SpaCy

**Вопросы:**

  4. Выберите все случаи, когда предложная фраза присоединяется к глаголу (verb), как в группе предложений (A) выше.
      - Создайте список для всех пар (tupples) вида (лемматизированный глагол, лемматизированный предлог) = (verb lemma, preposition lemma).
      - Посчитайте сколько таких случаев.
      - Выпишите 5 наиболее частотный пар (verb lemma, preposition lemma)
      - Подсказка: В spaCy можно получить зависимый токен  с помощью *token*.children, а тип зависимости с помощью *child.dep_*. 
  

In [22]:
verb_tokens = [token for token in tokens if token.pos_ == 'VERB']

In [23]:
verb_prep_list = []
for verb_token in verb_tokens:
    for child in verb_token.children:
        if child.dep_ == 'prep':
            verb_prep_list += [(verb_token.lemma_, child.lemma_)]

In [24]:
Counter(verb_prep_list).most_common(5)

[(('think', 'of'), 63),
 (('talk', 'of'), 54),
 (('come', 'to'), 50),
 (('go', 'to'), 43),
 (('see', 'in'), 41)]

  5. Сделайте тоже самое для случаев, когда предложная фраза присоединяется к дополнению глагола (verb's object), как в группе предложений (B). 
      - Создайте список для всех пар (tupples) вида (лемматизированный дополнение, лемматизированный предлог) = (verb's object lemma, preposition lemma).
      - Посчитайте сколько таких случаев.
      - Выпишите 5 наиболее частотный пар (verb's object, preposition lemma)?
  ?

In [25]:
verb_obj_prep_list = []
for verb_token in verb_tokens:
    for child in verb_token.children:
        if child.dep_ == 'dobj':
            for grandchild in child.children:
                if grandchild.dep_ == 'prep':
                    verb_obj_prep_list += [(child.lemma_, grandchild.lemma_)]

In [26]:
len(verb_obj_prep_list)

1034

In [27]:
Counter(verb_obj_prep_list).most_common(5)

[(('nothing', 'of'), 18),
 (('account', 'of'), 10),
 (('necessity', 'of'), 9),
 (('opinion', 'of'), 8),
 (('deal', 'of'), 8)]

  6. Постройте систему зависимостей для предложения "I shot an elephant in my pyjamas"
      - К чему присоединена предложная группа?
      - Кто был в пижаме с точки зрения SpaCy?

In [28]:
show_linkage("I shot an elephant in my pyjamas")

**Вывод**: предложная группа присоединена к сказуемуому, стрелял I (Я)

**Бонус:** Выберите несколько случайных предложений из корпуса. Постройте для них систему зависимостей. 
- Похоже на правду?

In [29]:
show_linkage(' '.join(sentences[21]))

In [30]:
show_linkage(' '.join(sentences[234]))

**Вывод** похоже на правду

Решение присылйте на почту  bdt-mf-ml-nlp-2020-q4@bigdatateam.org  

В теме письма укажите: ``HW2:CompLing. ФИО``

Пожалуйста, оставьте обратную связь о задании [по ссылке](http://rebrand.ly/mfnlp2020q4_feedback_hw02). Она (при желании) анонимна ;)